In [15]:
import os
root_directory = "./news_dataset"
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import hazm
import pandas as pd
import string
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score , classification_report

In [2]:
global_seen_docs = {}
untitled_folder_count = 0
unique_files = []
unique_files_text = []
unique_files_categories = {}
all_files = []
content_by_category = {}
for root, dirs, files in os.walk(root_directory):
    if os.path.basename(root) == "Untitled Folder":
        untitled_folder_count += 1
        continue
    relative_path = os.path.relpath(root, root_directory)
    path_components = ["Root"] if relative_path == "." else relative_path.split(os.sep)
    for component in path_components:
        if component == "Untitled Folder" or component == "Root":
            continue
        if component not in global_seen_docs.keys():
            global_seen_docs[component] = set()
            content_by_category[component] = {'titles': [], 'bodies': []}
    files.sort(key=lambda x: int(os.path.splitext(x)[0]))
    for file in files:
        all_files.append(file)
        file_path = os.path.join(root, file)
        all_categories = []
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            title = lines[0].strip()
            body = ''.join(lines[1:]).strip()
        for category in path_components:
            if category != "Untitled Folder":
                all_categories.append(category)
                if file not in global_seen_docs[category]:
                    global_seen_docs[category].add(file)
                    content_by_category[category]['titles'].append(title)
                    content_by_category[category]['bodies'].append(body)
        if file not in unique_files and len(all_categories) > 0:
            unique_files.append(file)
            unique_files_text.append(body)
            unique_files_categories[len(unique_files) -1]= list(set(all_categories))
        elif file in unique_files and len(all_categories) > 0:
            index_of_file = unique_files.index(file)
            last_categories = unique_files_categories[index_of_file]
            updated_categories = list(set(last_categories + all_categories))
            unique_files_categories[index_of_file] = updated_categories
            
            
            

In [3]:
print("Number of categories: " + str(len(global_seen_docs.values())))

Number of categories: 113


In [4]:
print("Number of documents: " + str(len(all_files)))

Number of documents: 17599


In [5]:
print("Number of unique documents: " + str(len(unique_files)))

Number of unique documents: 6539


In [6]:
data_samples = []
for category in content_by_category.keys():
    contents = content_by_category[category]
    title, body = contents['titles'][0],contents['bodies'][0]
    displayed_body = (body[:47] + '...') if len(body) > 50 else body
    data_samples.append({"Title": title, "Class": category, "Body": displayed_body})

df_samples = pd.DataFrame(data_samples)

display(df_samples)


,Title,Class,Body
0,مشاور اوباما: تجمع حوثی‌ها در صنعا را محکوم می...,اجتماعی,خبرگزاری تسنیم: مشاور ارشد رئیس جمهور آمریکا د...
1,مشاور اوباما: تجمع حوثی‌ها در صنعا را محکوم می...,اطلاعاتی,خبرگزاری تسنیم: مشاور ارشد رئیس جمهور آمریکا د...
2,مشاور اوباما: تجمع حوثی‌ها در صنعا را محکوم می...,جو فرهنگی حاکم بر کشور,خبرگزاری تسنیم: مشاور ارشد رئیس جمهور آمریکا د...
3,آمریکا پنج میلیارد رکورد اطلاعاتی ایرانی ها را...,فرهنگی امنیتی,تهران- ایرنا- اطلاعات حاکی از آن است که آمریکا...
4,آمریکا پنج میلیارد رکورد اطلاعاتی ایرانی ها را...,تهدیدات اجتماعی و فرهنگی,تهران- ایرنا- اطلاعات حاکی از آن است که آمریکا...
...,...,...,...
108,گروگانگیری در شعبه بانک انصار شوش با دستگیری گ...,مبارزه با گروگانگیری,دزفول- ایرنا- فرد مسلحی که صبح پنجشنبه با ورود...
109,هشدار رئیس‌جمهور چین نسبت به تهدید‌های فزاینده...,مسولان عالی تصمیم گیر کشوری و لشگری,رییس‌جمهوری چین با تاکید بر اینکه چین با تهدید...
110,آمریکا برای حمله به سوریه آماده می‌شود,پدافند هوایی,وزیر دفاع آمریکا اعلام کرد، پنتاگون در حال است...
111,پنتاگون در آستانه تعطیل دولت، ۵ میلیارد دلار ت...,عملیات روانی و تهاجم فرهنگی دشمن,ب پیش از فرستاده شدن ۸۰۰ هزار کارمند دولت آمری...


In [7]:
normalizer = hazm.Normalizer()
lemmatizer = hazm.Lemmatizer()
punctuations = [')', '(', '>', '<', "؛", "،", '{', '}', "؟", ':', "–", '»', '"', '«', '[', ']', '"', '+', '=', '?', '/',
                '//', '\\', '|', '!', '%', '&', '*', '$', '#', '؟', '*', '.', '_', '']
persian_numbers = '۰۱۲۳۴۵۶۷۸۹'
stopwords_list = hazm.stopwords_list()
preprocessed_category_docs = []
category_docs = []
for category in content_by_category.keys():
    category_text = ""
    for doc in content_by_category[category]['bodies']:
        normalized_text = normalizer.normalize(doc)
        tokens = hazm.word_tokenize(normalized_text)
        lemmatized_text = ""
        for token in tokens:
            if token in stopwords_list or token in punctuations or token.isdigit() or any(char in persian_numbers for char in token) or any(char in string.ascii_letters for char in token):
                continue
            lemmatized_token = lemmatizer.lemmatize(token)
            lemma = lemmatized_token.split('#')[1] if '#' in lemmatized_token else lemmatized_token
            lemmatized_text += lemma + " "
        category_text += lemmatized_text + " "

    preprocessed_category_docs.append(category_text.strip())
    category_docs.append(category)

In [8]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000,  ngram_range=(1,2),sublinear_tf=True)
tfidf_matrix_categories = tfidf_vectorizer.fit_transform(preprocessed_category_docs)
df_tfidf = pd.DataFrame(tfidf_matrix_categories.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

In [9]:
pd.set_option('display.max_colwidth', None) 
data_samples = []
for index, category in enumerate(category_docs):
    value = df_tfidf.iloc[index, :]
    contents = content_by_category[category]
    top_scores = value.sort_values(ascending=False).head(10)
    top_words = top_scores.index
    top_words_str = ', '.join(top_words)
    title = contents['titles'][0]
    data_samples.append({"Title": title, "Class": category, "Key words": top_words_str})

df_samples = pd.DataFrame(data_samples)
display(df_samples)

,Title,Class,Key words
0,مشاور اوباما: تجمع حوثی‌ها در صنعا را محکوم می‌کنیم/صنعا: از حمایت‌های آمریکا متشکریم,اجتماعی,"آسیب اجتماع, بچه پولدار, سبک زندگی, مخملی, جنگ نرم, سوروس, اعتیاد, پولدار, بافق, کودتا مخملی"
1,مشاور اوباما: تجمع حوثی‌ها در صنعا را محکوم می‌کنیم/صنعا: از حمایت‌های آمریکا متشکریم,اطلاعاتی,"یوتا, موادمخدر, جزوه, بلک واتر, ابوطالب, بچه پولدار, برنامه اقدام, سالیوان, واتر, آستارا"
2,مشاور اوباما: تجمع حوثی‌ها در صنعا را محکوم می‌کنیم/صنعا: از حمایت‌های آمریکا متشکریم,جو فرهنگی حاکم بر کشور,"بچه پولدار, پولدار, مشایی, انصار الله, عبدالملک الحوثی, جنبش انصار, حجاب, الحوثی, جمهور یمن, شهر صنعا"
3,آمریکا پنج میلیارد رکورد اطلاعاتی ایرانی ها را جمع آوری کرد,فرهنگی امنیتی,"آسیب اجتماع, بچه پولدار, مخملی, سبک زندگی, جنگ نرم, سوروس, اعتیاد, پولدار, کودتا مخملی, بافق"
4,آمریکا پنج میلیارد رکورد اطلاعاتی ایرانی ها را جمع آوری کرد,تهدیدات اجتماعی و فرهنگی,"آسیب اجتماع, بچه پولدار, اعتیاد, پولدار, تهاجم فرهنگ, موادمخدر, التحریر, نرخ بیکاری, ازدواج, سبک زندگی"
...,...,...,...
108,گروگانگیری در شعبه بانک انصار شوش با دستگیری گروگانگیر پایان یافت,مبارزه با گروگانگیری,"آزاد مرزبان, گروه جیش, جیش العدل, جمشید دانا, دانا فر, العدل, ذوالفقار, مرزبان ربوده, جمشید, مرزبان ایران"
109,هشدار رئیس‌جمهور چین نسبت به تهدید‌های فزاینده علیه امنیت ملی,مسولان عالی تصمیم گیر کشوری و لشگری,"فاینانس, انصار الله, عبدالملک الحوثی, آمرلی, جنبش انصار, دریابانی, مرزبان ناجا, قرارگاه پدافند, جمهور یمن, پوروشنکو"
110,آمریکا برای حمله به سوریه آماده می‌شود,پدافند هوایی,"قرارگاه پدافند, موشک اروپا, هوا خاتم, فرمانده قرارگاه, دادستان آلمان, پیامبر اعظم, ایران, خاتم الانبیاء, الانبیاء, سامانه رادار"
111,پنتاگون در آستانه تعطیل دولت، ۵ میلیارد دلار تجهیزات نظامی خرید,عملیات روانی و تهاجم فرهنگی دشمن,"جزوه, مدرسین, تهاجم فرهنگ, مشایی, بختیار, مصباح یزد, ایران, التحریر, حزب مشارکت, علمیه قم"


In [11]:
feature_names = tfidf_vectorizer.get_feature_names_out()
idf_scores = tfidf_vectorizer.idf_

# Pairing feature names with their IDF scores and sorting by IDF score in ascending order
sorted_features_by_idf = sorted(zip(feature_names, idf_scores), key=lambda x: x[1])

# Selecting the top N words with the lowest IDF scores
top_n = 100
lowest_idf_words = sorted_features_by_idf[:top_n]

# Creating a DataFrame from the top N lowest IDF scores
df_lowest_idf = pd.DataFrame(lowest_idf_words, columns=['Word', 'IDF Score'])

# Display the DataFrame
display(df_lowest_idf)

,Word,IDF Score
0,آزاد,1.0
1,آغاز,1.0
2,آماده,1.0
3,آمریکا,1.0
4,آمریکایی,1.0
...,...,...
95,تحلیلگر,1.0
96,تحولات,1.0
97,ترک,1.0
98,ترکیه,1.0


In [43]:
categories_doc_number = {}
forbidden_categories = []
for categories in unique_files_categories.values():
    for category in categories:
        if category in categories_doc_number.keys():
            categories_doc_number[category]+=1
        else:
            categories_doc_number[category]=1
for category,number in categories_doc_number.items():
    if number>1000:
        forbidden_categories.append(category)
    
for index, categories in list(unique_files_categories.items()):
    categories_set = set(categories)
    filtered_categories = categories_set - set(forbidden_categories)
    unique_files_categories[index] = list(filtered_categories)
print(unique_files_categories)

{0: ['اعتصابات و اعتراضهای سیاسی هدایت شده', 'مبارزه با تروریسم', 'جو فرهنگی حاکم بر کشور', 'سفرها و ملاقات ها', 'مسولان عالی تصمیم گیر کشوری و لشگری', 'فرهنگی اجتماعی', 'اجتماعی', 'فعالیت احزاب و گروه ها'], 1: ['جو فرهنگی حاکم بر کشور', 'بحران های اجتماعی', 'جاسوسی سیاسی', 'فرهنگی امنیتی', 'اجتماعی'], 2: ['اعتصابات و اعتراضهای سیاسی هدایت شده', 'تقویت و توسعه سازمان ها', 'جو فرهنگی حاکم بر کشور', 'فناوری', 'فرهنگی اجتماعی', 'اجتماعی'], 3: ['اعتصابات و اعتراضهای سیاسی هدایت شده', 'جو فرهنگی حاکم بر کشور', 'فرهنگی اجتماعی', 'اجتماعی'], 4: ['اعتصابات و اعتراضهای سیاسی هدایت شده', 'مبارزه با تروریسم', 'جو فرهنگی حاکم بر کشور', 'فعالیت های نظامی گروههای مسلح مخالف یا معاند نظام', 'جغرافیایی', 'جغرافیای انسانی کشور', 'فرهنگی اجتماعی', 'اجتماعی'], 5: ['سازمان ها و بنیاد های اجتماعی بحران ساز', 'جو فرهنگی حاکم بر کشور', 'فرهنگی امنیتی', 'اجتماعی'], 6: ['اعتصابات و اعتراضهای سیاسی هدایت شده', 'طرح ها و برنامه ها', 'مسولان عالی تصمیم گیر کشوری و لشگری', 'جو فرهنگی حاکم بر کشور', 'فرهنگی اجتماعی

In [37]:
preprocessed_docs = []
categories = []
for index,doc_text in enumerate(unique_files_text):
    normalized_text = normalizer.normalize(doc_text)
    tokens = hazm.word_tokenize(normalized_text)
    lemmatized_text = ""
    for token in tokens:
        if token in stopwords_list or token in punctuations or token.isdigit() \
           or any(char in persian_numbers for char in token) or any(char in string.ascii_letters for char in token):
            continue
        lemmatized_token = lemmatizer.lemmatize(token)
        lemma = lemmatized_token.split('#')[1] if '#' in lemmatized_token else lemmatized_token
        lemmatized_text += lemma + " "
           
    preprocessed_docs.append(lemmatized_text.strip())
    categories.append(unique_files_categories[index]) 

In [38]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2), sublinear_tf=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_docs)

df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())



In [39]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(categories)
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.2, random_state=42)
model = MultiOutputClassifier(MultinomialNB(), n_jobs=-1)
model.fit(X_train, y_train)

MultiOutputClassifier(estimator=MultinomialNB(), n_jobs=-1)

In [40]:
y_pred = model.predict(X_test)

model_acc = accuracy_score(y_test,y_pred)

print('nb test accuracy:', model_acc)


nb test accuracy: 0.0


one label prediction

In [45]:
print(y_test[0])
print(y_pred[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [42]:
import numpy as np

def calculate_precision_recall(y_true, y_pred):
    """
    Calculate precision and recall for positive labels in a multi-label setting.
    
    Parameters:
    - y_true: numpy array, true binary indicator matrix for labels.
    - y_pred: numpy array, predicted binary indicator matrix for labels.
    
    Returns:
    - precision: The precision score for positive labels.
    - recall: The recall score for positive labels.
    """
    TP = np.sum((y_pred == 1) & (y_true == 1))
    
    FP = np.sum((y_pred == 1) & (y_true == 0))
    
    FN = np.sum((y_pred == 0) & (y_true == 1))
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    

    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    
    return precision, recall



precision, recall = calculate_precision_recall(y_test, y_pred)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Precision: 0.5469
Recall: 0.0080


In [12]:

def label_based_accuracy(y_true, y_pred):
    """
    Compute label-based accuracy for multi-label classification.
    
    Parameters:
    - y_true: numpy array, true binary indicator matrix for labels.
    - y_pred: numpy array, predicted binary indicator matrix for labels.
    
    Returns:
    - accuracy: The average proportion of correctly predicted labels.
    """
    # Count of correctly predicted labels per sample
    correct_predictions_per_sample = np.sum(y_true == y_pred, axis=1)
    
    # Total number of labels per sample
    total_labels = y_true.shape[1]
    
    # Accuracy per sample
    sample_accuracies = correct_predictions_per_sample / total_labels
    
    # Overall accuracy is the mean of sample accuracies
    overall_accuracy = np.mean(sample_accuracies)
    
    return overall_accuracy

# Example usage
y_pred = model.predict(X_test)
accuracy = label_based_accuracy(y_test, y_pred)
print(f"Label-based Accuracy: {accuracy:.4f}")



Label-based Accuracy: 0.9627
